### Fetch the zip file

In [1]:
! wget https://databank.worldbank.org/data/download/WDI_csv.zip

--2022-02-02 17:59:00--  https://databank.worldbank.org/data/download/WDI_csv.zip
Resolving databank.worldbank.org (databank.worldbank.org)... 192.86.102.146
Connecting to databank.worldbank.org (databank.worldbank.org)|192.86.102.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67669068 (65M) [application/x-zip-compressed]
Saving to: ‘WDI_csv.zip.1’

WDI_csv.zip.1        21%[===>                ]  14,16M   654KB/s    in 23s     

2022-02-02 17:59:29 (639 KB/s) - Read error at byte 14843904/67669068 (Connection reset by peer). Retrying.

--2022-02-02 17:59:30--  (try: 2)  https://databank.worldbank.org/data/download/WDI_csv.zip
Connecting to databank.worldbank.org (databank.worldbank.org)|192.86.102.146|:443... connected.
HTTP request sent, awaiting response... 206 Partial Content
Length: 67669068 (65M), 52825164 (50M) remaining [application/x-zip-compressed]
Saving to: ‘WDI_csv.zip.1’

WDI_csv.zip.1        25%[++++>               ]  16,65M   623KB/s    eta

#### Unzip

In [ ]:
!unzip WDI_csv.zip -d data

Archive:  WDI_csv.zip
replace data/WDIData.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [1]:
# Install other libraries
!pip install pandas
!pip install ipython_sql

In [1]:
# Import libraries
import pandas as pd
import sqlite3
import numpy as np

In [2]:
# Load dataframes
df_data = pd.read_csv("data/WDIData.csv")
df_country = pd.read_csv("data/WDICountry.csv")
df_country_series = pd.read_csv("data/WDICountry-Series.csv")
df_WDIFootNote = pd.read_csv("data/WDIFootNote.csv")
df_series = pd.read_csv("data/WDISeries.csv")

df_data.columns = df_data.columns.str.replace(" ", "_")
df_data = df_data.drop(['Unnamed:_65'], axis=1, errors='ignore')

df_country.columns = df_country.columns.str.replace(" ", "_")
df_country_series.columns = df_country_series.columns.str.replace(" ", "_")
df_WDIFootNote.columns = df_WDIFootNote.columns.str.replace(" ", "_")
df_series.columns = df_series.columns.str.replace(" ", "_")
# df_series_time = pd.read_csv("data/WDISeries-Time.csv")

# df_series_time.to_sql('WDISeriesTime', conn, if_exists='replace')

In [3]:
df_country.head()

,Country_Code,Short_Name,Table_Name,Long_Name,2-alpha_code,Currency_Unit,Special_Notes,Region,Income_Group,WB-2_code,...,Government_Accounting_concept,IMF_data_dissemination_standard,Latest_population_census,Latest_household_survey,Source_of_most_recent_Income_and_expenditure_data,Vital_registration_complete,Latest_agricultural_census,Latest_industrial_data,Latest_trade_data,Unnamed:_30
0,ABW,Aruba,Aruba,Aruba,AW,Aruban florin,NaN,Latin America & Caribbean,High income,AW,...,NaN,Enhanced General Data Dissemination System (e-...,2020 (expected),NaN,NaN,Yes,NaN,NaN,2018.0,NaN
1,AFE,Africa Eastern and Southern,Africa Eastern and Southern,Africa Eastern and Southern,ZH,NaN,"26 countries, stretching from the Red Sea in t...",NaN,NaN,ZH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Afghanistan,Afghanistan,Islamic State of Afghanistan,AF,Afghan afghani,Fiscal year end: March 20; reporting period fo...,South Asia,Low income,AF,...,Consolidated central government,Enhanced General Data Dissemination System (e-...,1979,"Demographic and Health Survey, 2015","Integrated household survey (IHS), 2016/17",NaN,NaN,NaN,2018.0,NaN
3,AFW,Africa Western and Central,Africa Western and Central,Africa Western and Central,ZI,NaN,"22 countries, stretching from the westernmost ...",NaN,NaN,ZI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AGO,Angola,Angola,People's Republic of Angola,AO,Angolan kwanza,NaN,Sub-Saharan Africa,Lower middle income,AO,...,Budgetary central government,Enhanced General Data Dissemination System (e-...,2014,"Demographic and Health Survey, 2015/16","Integrated household survey (IHS), 2008/09",NaN,NaN,NaN,2018.0,NaN


In [4]:
df_series.head()

,Series_Code,Topic,Indicator_Name,Short_definition,Long_definition,Unit_of_measure,Periodicity,Base_Period,Other_notes,Aggregation_method,...,Notes_from_original_source,General_comments,Source,Statistical_concept_and_methodology,Development_relevance,Related_source_links,Other_web_links,Related_indicators,License_Type,Unnamed:_20
0,AG.AGR.TRAC.NO,Environment: Agricultural production,"Agricultural machinery, tractors",NaN,Agricultural machinery refers to the number of...,NaN,Annual,NaN,NaN,Sum,...,NaN,NaN,"Food and Agriculture Organization, electronic ...",A tractor provides the power and traction to m...,Agricultural land covers more than one-third o...,NaN,NaN,NaN,CC BY-4.0,NaN
1,AG.CON.FERT.PT.ZS,Environment: Agricultural production,Fertilizer consumption (% of fertilizer produc...,NaN,Fertilizer consumption measures the quantity o...,NaN,Annual,NaN,NaN,Weighted average,...,NaN,NaN,"Food and Agriculture Organization, electronic ...",Fertilizer consumption measures the quantity o...,"Factors such as the green revolution, has led ...",NaN,NaN,NaN,CC BY-4.0,NaN
2,AG.CON.FERT.ZS,Environment: Agricultural production,Fertilizer consumption (kilograms per hectare ...,NaN,Fertilizer consumption measures the quantity o...,NaN,Annual,NaN,NaN,Weighted average,...,NaN,NaN,"Food and Agriculture Organization, electronic ...",Fertilizer consumption measures the quantity o...,"Factors such as the green revolution, has led ...",NaN,NaN,NaN,CC BY-4.0,NaN
3,AG.LND.AGRI.K2,Environment: Land use,Agricultural land (sq. km),NaN,Agricultural land refers to the share of land ...,NaN,Annual,NaN,NaN,Sum,...,NaN,NaN,"Food and Agriculture Organization, electronic ...",Agricultural land constitutes only a part of a...,Agricultural land covers more than one-third o...,NaN,NaN,NaN,CC BY-4.0,NaN
4,AG.LND.AGRI.ZS,Environment: Land use,Agricultural land (% of land area),NaN,Agricultural land refers to the share of land ...,NaN,Annual,NaN,NaN,Weighted average,...,NaN,NaN,"Food and Agriculture Organization, electronic ...",Agriculture is still a major sector in many ec...,Agricultural land covers more than one-third o...,NaN,NaN,NaN,CC BY-4.0,NaN


In [5]:
df_data.head()

,Country_Name,Country_Code,Indicator_Name,Indicator_Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,15.981256,16.320475,16.643243,16.994695,17.313168,17.604950,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,28.381836,32.224027,32.046478,31.323579,33.312163,38.380433,39.754201,42.168241,43.640661,NaN
2,Africa Eastern and Southern,AFE,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,15.295950,20.525353,19.461383,17.790698,16.553470,23.907897,24.624725,26.813900,28.841150,NaN
3,Africa Eastern and Southern,AFE,"Access to electricity, urban (% of urban popul...",EG.ELC.ACCS.UR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,65.470615,66.303599,66.496010,65.828988,66.926692,68.722184,71.085418,71.994933,73.589886,NaN
4,Africa Eastern and Southern,AFE,Account ownership at a financial institution o...,FX.OWN.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df2 = df_data[df_data['Country_Name'].isin(["Upper middle income", "Lower middle income", "High income", "Low income"])]
c_names = ['Country_Name','Country_Code', 'Indicator_Name', 'Indicator_Code']

r = [str(i) for i in range(1960, 2000)]
df2.drop(r + ['2020'], axis=1, inplace=True)
df2 = df2.melt(id_vars=c_names, 
        var_name="Year", 
        value_name="Value")
df2 = df2.dropna()

# display(df2)
# display(df2.groupby(['Indicator_Name', 'Country_Name'])[['Value']].count())

filtered_indicators = df2.groupby(['Indicator_Name', 'Country_Name'])['Value'].count().reset_index()

# display(filtered_indicators)
# display(filtered_indicators[filtered_indicators['Country_Name'] == 'Low income'])

filtered_indicators = filtered_indicators[filtered_indicators['Value'] == 20]['Indicator_Name'].unique()

# display(filtered_indicators)

print(f"Selected indicators counts {len(filtered_indicators)}")
# print(filtered_indicators[:4])

df_series['CAN_BE_USED'] = np.where(df_series['Indicator_Name'].isin(filtered_indicators), True, False)

Selected indicators counts 647


/home/ash/.local/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [7]:
conn = sqlite3.connect('data/data.db')
df_data.to_sql('WDIData', conn, if_exists='replace', index=False)
df_country.to_sql('WDICountry', conn, if_exists='replace', index=False) 
df_country_series.to_sql('WDICountrySeries', conn, if_exists='replace', index=False) 
df_WDIFootNote.to_sql('WDIFootNote', conn, if_exists='replace', index=False) 
df_series.to_sql('WDISeries', conn, if_exists='replace', index=False)
conn.close()

In [8]:
df_data = pd.read_csv("data/WDIData.csv")
df_data.columns = df_data.columns.str.replace(" ", "_")

print(f"Step 0:{df_data.shape}")

# 1: Countries
df_data = df_data[df_data['Country_Name'].isin(countries)]
countries = df_country[df_country.Income_Group.notnull()]['Table_Name']

print(f"Step 1:{df_data.shape}")

# 2: Indicators
series = df_series[df_series.Topic.str.startswith('Environment:', na = False)]['Indicator_Name']
df_data = df_data[df_data['Indicator_Name'].isin(series)]

print(f"Step 2:{df_data.shape}")

with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    
    # r = np.range(1960, 2020)
    print(df_data.isnull().sum())
    
df_data

Step 0:(383838, 66)


NameError: name 'countries' is not defined

In [ ]:
years = [str(i) for i in list(range(1960, 2021))]
c = ['Country_Name', 'Country_Code',  'Indicator_Code']
with pd.option_context('display.max_rows', 10,
                       'display.max_columns', 10,
                       'display.precision', 3,
                       ):
    
    display(df_data.groupby(['Country_Name', 'Indicator_Name'])[years].count())